Deutsch-Jozsa Algorithm Implementation in Qiskit

In [1]:
# Install qiskit if not already installed
# !pip install qiskit qiskit-aer

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
import random

Block 2: Create Query (Query Gate) Functions

In [ ]:
def create_constant_oracle(n):
    """
    Creates a constant oracle that returns either 0 or 1 for all inputs.
    
    Args:
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Query circuit
    """
    oracle = QuantumCircuit(n + 1, name="Constant Query")
    
    # Randomly choose to return 0 (do nothing) or 1 (flip output qubit)
    output = random.randint(0, 1)
    
    if output == 1:
        oracle.x(n)  # Flip the output qubit
    
    return oracle, "constant"


def create_balanced_oracle(n):
    """
    Creates a balanced oracle that returns 0 for half inputs and 1 for other half.
    
    Args:
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Query circuit
    """
    oracle = QuantumCircuit(n + 1, name="Balanced Query")
    
    # Create a random balanced function
    # One simple approach: use CNOT gates controlled by random qubits
    b_str = "".join([random.choice(['0', '1']) for _ in range(n)])
    
    # Apply X gates based on the random binary string
    for i, bit in enumerate(b_str):
        if bit == '1':
            oracle.x(i)
    
    # Apply CNOT gates from each input qubit to output qubit
    for i in range(n):
        oracle.cx(i, n)
    
    # Reverse the X gates
    for i, bit in enumerate(b_str):
        if bit == '1':
            oracle.x(i)
    
    return oracle, "balanced"

Block 3: Create Random Query Function

In [3]:
def create_random_oracle(n):
    """
    Randomly creates either a constant or balanced oracle.
    
    Args:
        n: Number of input qubits
    
    Returns:
        tuple: (QuantumCircuit, oracle_type)
    """
    oracle_type = random.choice(['constant', 'balanced'])
    
    if oracle_type == 'constant':
        return create_constant_oracle(n)
    else:
        return create_balanced_oracle(n)

Block 4: Implement Deutsch-Jozsa Algorithm

In [4]:
def deutsch_jozsa_algorithm(oracle, n):
    """
    Implements the Deutsch-Jozsa algorithm.
    
    Args:
        oracle: Quantum circuit representing the query gate
        n: Number of input qubits
    
    Returns:
        QuantumCircuit: Complete Deutsch-Jozsa circuit
    """
    # Create quantum and classical registers
    qr = QuantumRegister(n + 1, 'q')
    cr = ClassicalRegister(n, 'c')
    dj_circuit = QuantumCircuit(qr, cr)
    
    # Step 1: Initialize the output qubit to |1⟩
    dj_circuit.x(n)
    
    # Step 2: Apply Hadamard gates to all qubits
    for i in range(n + 1):
        dj_circuit.h(i)
    
    dj_circuit.barrier()
    
    # Step 3: Apply the oracle
    dj_circuit.compose(oracle, inplace=True)
    
    dj_circuit.barrier()
    
    # Step 4: Apply Hadamard gates to input qubits
    for i in range(n):
        dj_circuit.h(i)
    
    dj_circuit.barrier()
    
    # Step 5: Measure the input qubits
    for i in range(n):
        dj_circuit.measure(i, i)
    
    return dj_circuit

Block 5: Function to Run and Analyze Results

In [6]:
def run_deutsch_jozsa(circuit, shots=1024):
    """
    Runs the Deutsch-Jozsa circuit and returns results.
    
    Args:
        circuit: Deutsch-Jozsa quantum circuit
        shots: Number of measurement shots
    
    Returns:
        dict: Measurement results
    """
    # Use AerSimulator
    simulator = AerSimulator()
    
    # Execute the circuit
    job = simulator.run(circuit, shots=shots)
    result = job.result()
    counts = result.get_counts()
    
    return counts


def interpret_results(counts, n):
    """
    Interprets the measurement results.
    
    Args:
        counts: Measurement results dictionary
        n: Number of input qubits
    
    Returns:
        str: 'constant' or 'balanced'
    """
    # Get the most common measurement result
    most_common = max(counts, key=counts.get)
    
    # If all zeros, function is constant
    if most_common == '0' * n:
        return "constant"
    else:
        return "balanced"

Block 6: Main Execution and Testing

In [13]:
def test_deutsch_jozsa(n=3):
    """
    Complete test of the Deutsch-Jozsa algorithm.
    
    Args:
        n: Number of input qubits (default: 3)
    """
    print(f"=" * 60)
    print(f"Testing Deutsch-Jozsa Algorithm with {n} input qubits")
    print(f"=" * 60)
    
    # Create a random oracle
    oracle, actual_type = create_random_oracle(n)
    
    print(f"\nActual Query Type: {actual_type}")
    print(f"\nQuery Circuit:")
    print(oracle.draw(output='text'))
    
    # Create the Deutsch-Jozsa circuit
    dj_circuit = deutsch_jozsa_algorithm(oracle, n)
    
    print(f"\nComplete Deutsch-Jozsa Circuit:")
    print(dj_circuit.draw(output='mpl'))
    
    # Run the circuit
    counts = run_deutsch_jozsa(dj_circuit)
    
    print(f"\nMeasurement Results:")
    print(counts)
    
    # Interpret results
    predicted_type = interpret_results(counts, n)
    
    print(f"\nPredicted Query Type: {predicted_type}")
    print(f"Actual Query Type: {actual_type}")
    
    if predicted_type == actual_type:
        print("✓ SUCCESS: Prediction matches actual query type!")
    else:
        print("✗ FAILED: Prediction does not match actual query type!")
    
    print(f"=" * 60)
    
    return dj_circuit, oracle, counts


# Run the test
if __name__ == "__main__":
    circuit, oracle, results = test_deutsch_jozsa(n=3)

Testing Deutsch-Jozsa Algorithm with 3 input qubits

Actual Query Type: balanced

Query Circuit:
                    
q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘

Complete Deutsch-Jozsa Circuit:
Figure(1123.61x451.5)

Measurement Results:
{'111': 1024}

Predicted Query Type: balanced
Actual Query Type: balanced
✓ SUCCESS: Prediction matches actual query type!
